# Advanced Lane Finding Project

The steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify (dt. ausrichten) binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

## Import Packages

In [1]:
#importing some useful packages

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import glob # glob API for importing list of (calibration) images
from line import * # import class Line() from line.py

## Camera calibration using chessboard images

In [2]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32) 
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2) # nx=9 and ny=6

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg') # importing the glob API

# print(images[0]) # DEBUG

# Step through the list and search for chessboard corners and if found, add object and image points (needed later in cv.calibrateCamera())
for fname in images:
    img = plt.imread(fname)
    # no cv2.undistort(img, mtx, dist, None) here because cal images are not distorted (only the first one a little bit maybe)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) # why? to find corners easily in the next step
                                                # if you are using cv2.imread() or the glob API, as happens here,
                                                # this will read in a BGR image and you should convert to grayscale
                                                # using cv2.COLOR_BGR2GRAY.

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None) # nx=9 and ny=6 (ret=return)

    # If found, add object points, image points
    if ret == True:
        #print(objp) #DEBUG
        objpoints.append(objp) # für jedes Bild in der for-Schleife wird dasselbe array (objp) hinzugefügt
        imgpoints.append(corners)

        # Draw and display the corners (this is not necessary but helpful for demonstration purposes)
        # img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        # plt.imshow(img)

# Perform camera calibration
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1:], None, None)

## Helper functions (for both pipelines)

In [ ]:
# function that takes an image, object points, and image points, performs the camera calibration, image distortion correction
# and returns the undistorted image
def cal_undistort(img, objpoints, imgpoints):
    undist = cv2.undistort(img, mtx, dist, None, mtx)  # undist war im Kurs dst (destination)
    return undist

def warp(img, nx, ny, mtx, dist): # to warp >> dt. (ver-)drehen, warped
    # define 4 source points src = np.float32([[,],[,],[,],[,]])
    # Note: you could pick any four of the detected corners as long as those four corners define a rectangle.
    # One especially smart way to do this would be to use four well-chosen corners that were automatically detected
    # during the undistortion steps.
    
    
    src = np.float32([[  100.,   719.],  # Links unten
                      [  542.,   470.],  # Links oben
                      [  738.,   470.],  # Rechts oben
                      [ 1180.,   719.]]) # Rechts unten
    
    
    '''
    source points for A99Munich test videos
    src = np.float32([[  235.,   719.], 
                      [  580.,   536.],
                      [  688.,   536.],
                      [ 1003.,   719.]])
    '''
        
    # define 4 destination points dst = np.float32([[,],[,],[,],[,]])
    
    dst = np.float32([[ 200.,  720.],
                      [ 200.,    0.],
                      [ 1080.,    0.],
                      [ 1080.,  720.]])
    
        
    # use cv2.getPerspectiveTransform() to get M, the transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    
    # use cv2.warpPerspective() to warp your image to a top-down view
    img_size = (img.shape[1], img.shape[0])
    warped = cv2.warpPerspective(img, M, img_size)
        
    return warped, M, Minv

def threshold(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    # Note: img is the undistorted image
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    # Grayscale image? No! Why? Single color needed to calculate derivatives >> "single color" can also mean h-/l- or s-channel! 
    # We already saw that standard grayscaling lost color information for the lane lines. This is while we do not grayscale.
    # Exploring gradients in other colors spaces / color channels shows that l-channel works better
        
    # Sobel x (identifies vertical edges)
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient (Sobel x) # threshold means as much as creating a binary by means of thresholds
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    # Stack each channel to view their individual contributions in green and blue respectively
    # This returns a stack of the two binary images, whose components you can see as different color
    # The green is the gradient threshold component and the blue is the color channel threshold component
    color_binary = np.dstack((np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    
    return color_binary

def find_lane_pixels(binary_warped):
    
    '''
    Findet mit der sliding windows Methode die Pixel im binary_warped, die zu den Spurlinien gehören und gibt sie wie folgt aus:
    
    leftx >> [308 309 310 ...362 363 364] Liste der x-Koordinaten von Pixeln, die zur left lane gehören müssten
    lefty >> [640 640 640 ... 159 159 159] List der y-Koordinaten von Pixeln, die zur right lane gehören müssten
    rightx >> []
    righty >> []
    
    Außerdem werden auf das Input-Bild die sliding windows (grüner Rahmen) gezeichnet.
    
    '''
    
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        #print("left_lane_inds: ", left_lane_inds, "Liste der left lane pixel indices (nonzero pixels in x and y within the window)")
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    print("leftx: ", leftx)
    lefty = nonzeroy[left_lane_inds]
    print("lefty: ", lefty)
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped):
    
    '''
    Die Polynomfunktion wird ermittelt (np.polyfit()) und die sich daraus ergebende Linie gelb eingezeichnet in das Input-image.
    Außerdem werden die zu Beginn der function ermittelten lane pixels rot (left lane) bzw. blau (right lane) eingefärbt.
    (Das Zeichnen der windows (grüner Rahmen) geschieht in der function find_lane_lines, die die sliding windows Methode nutzt.)
    
    
    Mit Hilfe der Pixel, die zu den lane lines gehören müssten (diese werden durch Aufruf der Funktion find_lane_lines()
    in dieser function nochmals separat ermittelt; Ergebnis z.B. leftx >> Liste mit den x-Koordinaten der left lane),
    wird eine passende Polynomfunktion x = A*y**2 + B*y + C ermittelt ("gefittet"). Dabei sind:
        
        left_fit >> [A B C] Polynomkoeffizienten
        right_fit >> dito
        
        ploty >> z.B. [0 1 2 3 4 ... 717 718 719] y-Koordinaten des Polynoms bzw. der sich daraus ergebenden Linie
        
        left_fitx >> z.B. [375 374,8 374,65 374,48 ... 319] x-Koordinaten des Polynoms bzw. der sich daraus ergebenden Kurve
                  >> errechnet sich aus A*y**2 + B*y + C mit y = ploty
        right_fitx >> dito
        
    '''
    
    
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2) # left_fit beinhaltet die Koeffizienten A, B und C
    print("left_fit: ", left_fit, "Dies sind die Koeffizienten A, B und C")
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0]) # ploty ist eine Liste von [0 1 2 .... 719]
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2] # left_fitx ist die gesamte Funktion (x = A*y**2 + B*y + C) des gefundenen, passenden Polynoms "fit"
        print("left_fitx: ", left_fitx)
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0] # Pixel der left lane werden rot dargestellt, Pixel der right lane blau
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    plt.plot(left_fitx, ploty, color='yellow') # x-Koordinaten des Polynoms (left_fitx) und y-Koordinaten des Polynoms (ploty) zum Plotten
    plt.plot(right_fitx, ploty, color='yellow')

    #return out_img
    return out_img, left_fit, right_fit
    
def fit_poly(img_shape, leftx, lefty, rightx, righty):
    
    '''
    Was ist der Unterschied zur function fit_polynomial()?
    Gemeinsamkeit: Auch hier wird wieder die Polynomfunktion mit np.polyfit() ermittelt: z.B. left_fit >> [A B C] Polynomkoeff. 
    Auch hier werden wieder x- und y-Werte generiert, um sie zu plotten, z.B. ploty
    
    Der entscheidende Unterschied ist der Input der Funktion! Kein Bild wird an die Funktion übergeben sondern direkt die Werte,
    die sich aus der function find_lane_pixels() ergeben. 
    Neue Idee: Wieso nicht innerhalb dieser function diese Werte durch Aufrufen von find_lane_pixels() ermitteln?
    
    Auch der Output ist anders. 
    
    '''
    
    
    # Fit a second order polynomial to each line with np.polyfit()
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    
    # Calc both polynomials using ploty, left_fit and right_fit
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty

def search_around_poly(binary_warped, left_fit, right_fit):
    
    # width of the margin around the previous polynomial to search
    margin = 50

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    #Set the area of search based on activated x-values within the +/- margin of our polynomial function
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    ## End visualization steps ##
    
    return result

def measure_curvature_real(ploty, left_fit, right_fit):
    
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit[0]*y_eval*ym_per_pix + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval*ym_per_pix + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    
    return left_curverad, right_curverad

## Pipeline (test images)

### Read in an image

In [ ]:
#reading in an image
#img = mpimg.imread('test_images/signs_vehicles_xygrad.png')
#img = mpimg.imread('test_images/straight_lines1.jpg')
#img = mpimg.imread('test_images/straight_lines2.jpg')
img = mpimg.imread('test_images/test1.jpg')
#img = mpimg.imread('test_images/test2.jpg')
#img = mpimg.imread('test_images/test3.jpg')
#img = mpimg.imread('test_images/test4.jpg')
#img = mpimg.imread('test_images/test5.jpg')
#img = mpimg.imread('test_images/test6.jpg')
#img = mpimg.imread('test_images/test_A99Munich_1.jpg')

# ATTENTION: If you are using cv2.imread() or the glob API, this will read in a BGR image and you should convert to grayscale
# (in later image processing steps) using cv2.COLOR_BGR2GRAY.

#printing out some stats and plotting
print('This image is:', type(img), 'with dimensions:', img.shape)
plt.imshow(img)  # single color channel image, e.g. gray , call as plt.imshow(gray, cmap='gray')

### Distortion correction

In [ ]:
undist = cal_undistort(img, objpoints, imgpoints)
plt.imshow(undist)

### Color & Gradient Threshold

In [ ]:
thresholded = threshold(undist)
plt.imshow(thresholded)

### Perspective Transform to "birds-eye view"

In [ ]:
top_down, perspective_M, perspective_Minv = warp(thresholded, 9, 6, mtx, dist)
plt.imshow(top_down)

### Detect lane pixels implementing sliding windows and fit a polynomial to find the lane boundary

In [ ]:
binary_warped = cv2.cvtColor(top_down, cv2.COLOR_BGR2GRAY) # binary_warped entspricht top_down in grau
left_lane = Line() # create/initiate object called left_lane with class Line() from line.py
right_lane = Line()
out_img = fit_polynomial(binary_warped)
plt.imshow(out_img)

### Skip the sliding windows step once you've found the lines / Search from prior

In [ ]:
# Polynomial fit values from the previous frame
# Make sure to grab the actual values from the previous step in your project
leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped) # muss hier für die nächsten beiden Schritte errechnet werden
left_fit = np.polyfit(lefty, leftx, 2) # muss hier nochmal explizit errechnet werden, da es vorher nur innerhalb fit_polynomial() berechnet wurde (lokale Variable)
right_fit = np.polyfit(righty, rightx, 2) # dito

# Run image through the pipeline
out_img2 = search_around_poly(binary_warped, left_fit, right_fit)

# View your output
plt.imshow(out_img2)

### Determine the curvature of the lane and vehicle position with respect to center

In [ ]:
# Calculate the radius of curvature in meters for both lane lines
left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty) # only calculated for ploty 
left_curverad, right_curverad = measure_curvature_real(ploty, left_fit, right_fit)

print(left_curverad, 'm', right_curverad, 'm')

### Drawing

In [ ]:
# Create an image to draw the lines on
warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

# Recast the x and y points into usable format for cv2.fillPoly()
pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
pts = np.hstack((pts_left, pts_right))

# Draw the lane onto the warped blank image
cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

# Warp the blank back to original image space using inverse perspective matrix (Minv)
Minv = perspective_Minv
newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
# Combine the result with the original image
result2 = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
plt.imshow(result2)

### Optional 2: Total pipeline in one code snippet

In [ ]:
#reading in an image
img = mpimg.imread('test_images/test1.jpg')

undist = cal_undistort(img, objpoints, imgpoints)

thresholded = pipeline(undist)

top_down, perspective_M, perspective_Minv = warp(thresholded, 9, 6, mtx, dist)

binary_warped = cv2.cvtColor(top_down, cv2.COLOR_BGR2GRAY) # binary_warped entspricht top_down in grau

out_img, left_fit, right_fit = fit_polynomial(binary_warped)

# Load our image - this should be a new frame since last time!
binary_warped = mpimg.imread('warped_example.jpg') # Wie kann man den nächsten Frame laden?

# Polynomial fit values from the previous frame already loaded in line 12

# Run image through the pipeline (Note that in your project, you'll also want to feed in the previous fits)
result = search_around_poly(binary_warped)

# View your output
plt.imshow(result)

### Optional: Total pipeline in one code snippet

In [ ]:
### Start of pipeline

#reading in an image
#img = mpimg.imread('test_images/signs_vehicles_xygrad.png')
#img = mpimg.imread('test_images/straight_lines1.jpg')
#img = mpimg.imread('test_images/straight_lines2.jpg')
#img = mpimg.imread('test_images/test1.jpg')
#img = mpimg.imread('test_images/test2.jpg')
#img = mpimg.imread('test_images/test3.jpg')
#img = mpimg.imread('test_images/test4.jpg')
#img = mpimg.imread('test_images/test5.jpg')
#img = mpimg.imread('test_images/test6.jpg')
img = mpimg.imread('test_images/test_A99Munich_1.jpg')

# distortion correction
undist = cal_undistort(img, objpoints, imgpoints)

# color & gradient thresholding
thresholded = pipeline(undist)

# perspective transform to "birds-eye view"
top_down, perspective_M, perspective_Minv = warp(thresholded, 9, 6, mtx, dist)

# Detect lane pixels implementing sliding windows and fit a polynomial to find the lane boundary
binary_warped = cv2.cvtColor(top_down, cv2.COLOR_BGR2GRAY) # binary_warped entspricht top_down in grau
out_img = fit_polynomial(binary_warped)

# Skip the sliding windows step once you've found the lines / Search from prior
leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped) 
left_fit = np.polyfit(lefty, leftx, 2)
right_fit = np.polyfit(righty, rightx, 2) # dito

# Run image through the pipeline
out_img2 = search_around_poly(binary_warped, left_fit, right_fit)

# Create an image to draw the lines on
warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

# Recast the x and y points into usable format for cv2.fillPoly()
pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
pts = np.hstack((pts_left, pts_right))

# Draw the lane onto the warped blank image
cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

# Warp the blank back to original image space using inverse perspective matrix (Minv)
Minv = perspective_Minv
newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
# Combine the result with the original image
result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
plt.imshow(result)


### End of pipeline

# Plot the result (original image and processed image)
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()

ax1.imshow(img)
ax1.set_title('Original Image', fontsize=40)

ax2.imshow(result)
ax2.set_title('Pipeline Result', fontsize=40)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Calculate the radius of curvature in meters for both lane lines
left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
left_curverad, right_curverad = measure_curvature_real(ploty, left_fit, right_fit)

print(left_curverad, 'm', right_curverad, 'm')

## Pipeline (video)

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    '''
    total pipeline from above as a function
    '''
    # distortion correction
    undist = cal_undistort(image, objpoints, imgpoints)

    # color & gradient thresholding
    thresholded = pipeline(undist)

    # perspective transform to "birds-eye view"
    top_down, perspective_M, perspective_Minv = warp(thresholded, 9, 6, mtx, dist)

    # Detect lane pixels implementing sliding windows and fit a polynomial to find the lane boundary
    binary_warped = cv2.cvtColor(top_down, cv2.COLOR_BGR2GRAY) # binary_warped entspricht top_down in grau
    out_img = fit_polynomial(binary_warped)

    # Skip the sliding windows step once you've found the lines / Search from prior
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped) 
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2) # dito

    # Run image through the pipeline
    out_img2 = search_around_poly(binary_warped, left_fit, right_fit)
    
    left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    Minv = perspective_Minv
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    
    return result

### Testing on project video:

In [ ]:
output = 'my_output1.mp4'
## To speed up the testing process, try pipeline on a shorter subclip of the video (.subclip())
#clip1 = VideoFileClip("project_video.mp4").subclip(0,5) #.subclip(start_second,end_second)
clip1 = VideoFileClip("project_video.mp4")
clip = clip1.fl_image(process_image) #NOTE: this function expects color images
%time clip.write_videofile(output, audio=False)

Play the video inline:

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))

### Testing on challenge video

In [ ]:
output2 = 'my_output2.mp4'
## To speed up the testing process, try pipeline on a shorter subclip of the video (.subclip())
clip1 = VideoFileClip("test_videos/challenge_video.mp4")
clip = clip1.fl_image(process_image)
%time clip.write_videofile(output2, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output2))

### Testing on harder challenge video

In [ ]:
output3 = 'my_output3.mp4'
## To speed up the testing process, try pipeline on a shorter subclip of the video (.subclip())
#clip1 = VideoFileClip("harder_challenge_video.mp4").subclip(0,5) #.subclip(start_second,end_second)
clip1 = VideoFileClip("harder_challenge_video.mp4")
clip = clip1.fl_image(process_image) #NOTE: this function expects color images
%time clip.write_videofile(output3, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output3))

### Testing on german highway A99 in Munich (short version)

In [ ]:
output4 = 'A99Munich_short_output.mp4'
clip1 = VideoFileClip("test_videos/A99Munich_short.mp4")
clip = clip1.fl_image(process_image) #NOTE: this function expects color images
%time clip.write_videofile(output4, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output4))

### Testing on german highway A99 in Munich (long version)

In [ ]:
output5 = 'A99Munich_long_output.mp4'
clip1 = VideoFileClip("test_videos/A99Munich_long.mp4").subclip(83,103)
clip = clip1.fl_image(process_image) #NOTE: this function expects color images
%time clip.write_videofile(output5, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output5))

In [ ]:
output6 = 'A99Munich_long_output_83-173.mp4'
clip1 = VideoFileClip("test_videos/A99Munich_long.mp4").subclip(83,173)
clip = clip1.fl_image(process_image) #NOTE: this function expects color images
%time clip.write_videofile(output6, audio=False)

In [ ]:
output7 = 'A99Munich_long_output_108-173.mp4'
clip1 = VideoFileClip("test_videos/A99Munich_long.mp4").subclip(108,173)
clip = clip1.fl_image(process_image) #NOTE: this function expects color images
%time clip.write_videofile(output7, audio=False)

## Issues Discussion

### Description of Problems

### Solutions to the problems